In [ ]:
import numpy as np

In [ ]:
train_file=open("Data/RU/train","r")
train_words=[]
tags=[]
for l in train_file:
    if l!="\n":
        lst=l.split()
        x=""
        for i in range(len(lst)-1):
            x+=lst[i]+" "
        x=x[0:-1]
        y=lst[-1]
        train_words.append(x)
        tags.append(y)

In [ ]:
test_file=open("Data/RU/dev.in","r")
test_words=[]
for l in test_file:
    if l!="\n":
        test_words.append(l[0:-1])

In [ ]:
def transition_helper(u,v,tags):
    count_u=0
    count_u_to_v=0
    for i in range(len(tags)):
        if tags[i]==u:
            count_u+=1
            if tags[i+1]==v:
                count_u_to_v+=1
    return count_u_to_v/count_u

def transition(tags):
    tags.insert(0,"START")
    tags.append("STOP")
    transition_parameters={}
    for i in range(len(tags)):
        if tags[i]=="STOP":
            break
        if (tags[i],tags[i+1]) not in transition_parameters:
            transition_parameters[(tags[i],tags[i+1])]=transition_helper(tags[i],tags[i+1],tags)
    del tags[0]
    del tags[-1]
    return transition_parameters

In [ ]:
def emission_helper(x, y,tags, words=None, k=1):
    count_y=0
    if x=="#UNK#":
        for i in tags:
            if i==y:
                count_y+=1
        return k/(count_y+k)
    count_y_to_x=0
    for i in range(len(tags)):
        if tags[i] == y:
            count_y += 1
            if words[i] == x:
                count_y_to_x += 1
    return (count_y_to_x) / (count_y + k)

def emission(tags,train_words,test_words):
    emission_word_tag={}
    for x in np.unique(np.array(test_words)):
        y_val={}
        for y in np.unique(np.array(tags)):
            if x in train_words:
                ep = emission_helper(x, y,tags,train_words)
                y_val.update({y:ep})
            else:
                ep = emission_helper("#UNK#", y,tags)
                y_val.update({y:ep})
        emission_word_tag[x] = y_val
    return emission_word_tag

In [ ]:
emission_word_tag=emission(tags,train_words,test_words)
transition_parameters=transition(tags)

print(emission_word_tag)
print(transition_parameters)

In [ ]:
import numpy as np

def viterbi(sequence, tags, transition_parameters, emission_parameters):
    n = len(sequence)
    num_tags = len(tags)

    LTR=np.zeros((num_tags,n))
    RTL=np.zeros((num_tags,n))


    first_word=sequence[0]
    for i in range(num_tags):
        if ("START", tags[i]) not in transition_parameters:
            LTR[i, 0] = 1e-10*emission_parameters[first_word][tags[i]]
        else:
            LTR[i, 0] = transition_parameters[("START", tags[i])] * emission_parameters[first_word][tags[i]]

    for w in range(1, n):
        for i in range(0,num_tags):
            max_prob = -1
            max_backpointer = -1
            for j in range(0,num_tags):
                if (tags[j],tags[i]) not in transition_parameters:
                    prob = LTR[j, w-1] * 1e-10 * emission_parameters[sequence[w]][tags[i]]
                else:
                    prob = LTR[j, w-1] * transition_parameters[(tags[j], tags[i])] * emission_parameters[sequence[w]][tags[i]]

                if prob > max_prob:
                    max_prob = prob
                    max_backpointer = j
            
            LTR[i, w] = max_prob
            RTL[i, w] = max_backpointer
    
    stop_max_prob = -1
    stop_max_backpointer = -1
    for i in range(num_tags):
        if (tags[i],"STOP") not in transition_parameters:
            prob = LTR[i, n-1] * 1e-10
        else:
            prob = LTR[i, n-1] * transition_parameters[(tags[i], "STOP")]

        if prob > stop_max_prob:
            stop_max_prob = prob
            stop_max_backpointer = i
    
    # Retrieve the best path using backpointers
    best_path = []
    for w in range(n-1,-1,-1):
        best_path.insert(0,tags[stop_max_backpointer])
        if w==0:
            break
        stop_max_backpointer=int(RTL[stop_max_backpointer,w])

    return best_path


In [ ]:
test_file=open("Data/RU/dev.in","r")
pred_output=open("DATA/RU/dev.p2.out","w")
sequence=[]

for l in test_file:
    if l!="\n":
        sequence.append(l[0:-1])
    else:
        predicted_tags=viterbi(sequence,np.unique(tags),transition_parameters,emission_word_tag)
        for i in range(0,len(predicted_tags)):
            pred_output.write(sequence[i]+" "+predicted_tags[i]+"\n")
        pred_output.write(l)
        pred_output.flush()
        sequence=[]